# Visualize CryinGAN training

In [ ]:
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import ase
from ase.io import read, write
from ase.visualize import view

In [ ]:
all_runs = pd.read_csv("../CryinGAN_COPY/losses.csv") # For square lattice
all_runs = pd.read_csv("../CryinGAN/losses.csv") # For full scale test

run_starts = all_runs["epoch"] == "epoch"

# Filter out the rows that are not part of the runs
# all_runs = all_runs[~run_starts]

# Add "run" column
all_runs["run"] = 0

run_idx = 0

for i in range(1,len(all_runs)):
    if run_starts[i]:
        all_runs.loc[i:, "run"] = run_idx
        run_idx += 1
    else:
        all_runs.loc[i, "run"] = run_idx

# Remove the first row of each run
all_runs = all_runs[~run_starts]
all_runs.set_index(["run", "epoch"], inplace=True)
all_runs = all_runs.astype(float)

# Read and visualize losses csv

run_idx = 0 # CHANGE THIS TO SELECT A RUN
df = all_runs.loc[run_idx]

fig = plt.figure(figsize=(20,10))
sns.lineplot(data=df, figure=fig, marker="o")
plt.grid(which="both")

In [ ]:
fig = plt.figure(figsize=(20,10))
sns.lineplot(data=df[["cost_gen", "D_fake"]],figure=fig, marker="o")

## Check on the generated structures

Numpy way from training-time generation

In [ ]:
ref_struct = read("../data/reference/cryingan_train.extxyz", index=1, format="extxyz")
ref_struct = read("../data/processed/crystal/Sq/phi-0.84/disorder-0.2", index=1, format="extxyz")
ref_struct = read("../data/processed/samples/phi-0.84", index=1, format="extxyz")

view(ref_struct, viewer="x3d", show_unit_cell=True, repeat=(1,1,1))

In [ ]:
import numpy as np
file = "../CryinGAN_COPY/gen_data/gen_coords_625.npy"
file = "../CryinGAN/gen_data/gen_coords_130.npy"
np_coords = np.load(file)
atoms = ref_struct.copy()
atoms.set_scaled_positions(np_coords[0,0])
view(atoms, viewer="x3d", show_unit_cell=True, repeat=(1,1,1))

In [ ]:
file = "../data/reference/cryingan_train.extxyz" # For reference data
file = "../data/processed/samples/phi-0.84" # For reference data
file = "../CryinGAN_COPY/gen.extxyz" # For generated data
file = "../CryinGAN/gen.extxyz" # For generated data

all_atoms = read(file, index=":", format="extxyz")
print("Read {} atoms".format(len(all_atoms)))
atoms = all_atoms[10] # For reference data

view(atoms, viewer="x3d", show_unit_cell=True, repeat=(1,1,1))

# Next steps

1. Provide Anshul with generated samples (50 each) in our data format
   1. Low packing fraction 0.70
   2. Mid 0.78
   3. High packing fraction 0.84
   4. Very high 0.86
2. Special assignment: Try out different loss functions to make the generated structures more physically feasible (or something else)
   1. Adding the bond distance discriminator
   2. Radius / overlap loss
   3. NN distance based loss
   4. Hexatic (k=5) order loss
   5. Other physical 
3. (Later: Conditioning on phi / other descriptors)

In [ ]:
all_atoms[i].get_cell()

In [ ]:
# Save the coordinates to a CSV file

import os
# os.mkdir("../data/gen")
# os.mkdir("../data/gen/phi-0.84")

for i in range(len(all_atoms)):
    with open(f"../data/gen/phi-0.84/sample-{i}", "w+") as f:
        
        atoms = all_atoms[i]

        L = atoms.get_cell()[0][0]
        N = len(atoms)

        header = "\t{N}\t{L}\t1626.81570886301\t\n".format(N=N, L=L)
        f.write(header)

        _df = pd.DataFrame({
            "class": atoms.get_atomic_numbers(),
            "x": atoms.get_positions()[:,0],
            "y": atoms.get_positions()[:,1],
        }).to_csv(f, index=False, header=False, sep="\t")
